# Real Estate Analysis

## Importing Libraries and Datasets

In [251]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import mysql.connector
import sqlite3
from dotenv import load_dotenv
import matplotlib.pyplot as plt

In [252]:
# Creating dataframes
agents_df = pd.read_json('Datasets/agents_cleaned.json')
buyers_df = pd.read_json('Datasets/buyers_cleaned.json')
listings_df = pd.read_json('Datasets/listings_final_expanded.json')
prop_att_df = pd.read_json('Datasets/property_attributes_final_expanded.json')
sales_df = pd.read_csv('Datasets/sales_cleaned.csv')

## Data Preparation and Cleaning

In [253]:
# Percentage of missing values
def per_miss_val(df):
  return 100 * df.isnull().sum() / len(df)

# Convert bool to int
def bool_to_int(x):
  return x.map({True: 1, False: 0})

### Agents

In [254]:
# First 5 rows
agents_df.head()

,Agent_ID,Name,Phone,Email,commission_rate,deals_closed,rating,experience_years,avg_closing_days
0,A0001,Agent A0001,+1-534-665-8373,a0001@realestate.com,2.00,52,4.3,10,64
1,A0002,Agent A0002,+1-493-463-4698,a0002@realestate.com,2.20,26,3.8,20,82
2,A0003,Agent A0003,+1-290-534-1121,a0003@realestate.com,2.85,297,3.6,21,83
3,A0004,Agent A0004,+1-691-610-4878,a0004@realestate.com,1.67,31,4.5,5,61
4,A0005,Agent A0005,+1-829-613-5411,a0005@realestate.com,1.11,198,4.3,16,67


In [255]:
# Last 5 rows
agents_df.tail()

,Agent_ID,Name,Phone,Email,commission_rate,deals_closed,rating,experience_years,avg_closing_days
45,A0046,Agent A0046,+1-873-939-2900,a0046@realestate.com,1.57,135,3.2,10,58
46,A0047,Agent A0047,+1-153-271-8602,a0047@realestate.com,1.63,91,4.0,16,33
47,A0048,Agent A0048,+1-508-587-5937,a0048@realestate.com,1.34,266,4.0,21,15
48,A0049,Agent A0049,+1-587-864-9724,a0049@realestate.com,1.70,17,4.1,14,33
49,A0050,Agent A0050,+1-339-488-7390,a0050@realestate.com,2.56,109,3.5,11,68


In [256]:
# Basic Information
agents_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Agent_ID          50 non-null     object 
 1   Name              50 non-null     object 
 2   Phone             50 non-null     object 
 3   Email             50 non-null     object 
 4   commission_rate   50 non-null     float64
 5   deals_closed      50 non-null     int64  
 6   rating            50 non-null     float64
 7   experience_years  50 non-null     int64  
 8   avg_closing_days  50 non-null     int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 3.6+ KB


In [257]:
# Shape
agents_df.shape

(50, 9)

In [258]:
# Removing "Name" column
agents_df.drop(columns='Name', inplace=True)
agents_df.head()

,Agent_ID,Phone,Email,commission_rate,deals_closed,rating,experience_years,avg_closing_days
0,A0001,+1-534-665-8373,a0001@realestate.com,2.00,52,4.3,10,64
1,A0002,+1-493-463-4698,a0002@realestate.com,2.20,26,3.8,20,82
2,A0003,+1-290-534-1121,a0003@realestate.com,2.85,297,3.6,21,83
3,A0004,+1-691-610-4878,a0004@realestate.com,1.67,31,4.5,5,61
4,A0005,+1-829-613-5411,a0005@realestate.com,1.11,198,4.3,16,67


In [259]:
# Formatting the phone number
from re import split

def format_number(st):
  split_lst = st.split('-')
  country_code = split_lst[0].replace('+','')
  phone_number = ''
  for i in range(1, len(split_lst)):
    phone_number += split_lst[i]

  return country_code, phone_number

agents_df['Country Code'] = agents_df.Phone.apply(lambda x: format_number(x)[0]).astype(int)
agents_df['Phone'] = agents_df.Phone.apply(lambda x: format_number(x)[1]).astype(int)
agents_df = agents_df[
  ['Agent_ID', 'Email', 'Phone', 'Country Code', 'commission_rate', 'deals_closed', 'rating', 'experience_years', 'avg_closing_days']
  ]
agents_df.head()


,Agent_ID,Email,Phone,Country Code,commission_rate,deals_closed,rating,experience_years,avg_closing_days
0,A0001,a0001@realestate.com,5346658373,1,2.00,52,4.3,10,64
1,A0002,a0002@realestate.com,4934634698,1,2.20,26,3.8,20,82
2,A0003,a0003@realestate.com,2905341121,1,2.85,297,3.6,21,83
3,A0004,a0004@realestate.com,6916104878,1,1.67,31,4.5,5,61
4,A0005,a0005@realestate.com,8296135411,1,1.11,198,4.3,16,67


In [260]:
# Missing values
per_miss_val(agents_df)

Agent_ID            0.0
Email               0.0
Phone               0.0
Country Code        0.0
commission_rate     0.0
deals_closed        0.0
rating              0.0
experience_years    0.0
avg_closing_days    0.0
dtype: float64

In [261]:
# Summary Statistics
agents_df.describe(include='all')

,Agent_ID,Email,Phone,Country Code,commission_rate,deals_closed,rating,experience_years,avg_closing_days
count,50,50,5.000000e+01,50.0,50.000000,50.000000,50.000000,50.000000,50.000000
unique,50,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,A0001,a0001@realestate.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,5.872193e+09,1.0,2.051400,136.640000,3.944000,14.780000,55.680000
std,NaN,NaN,2.638246e+09,0.0,0.513996,91.190708,0.575046,6.516196,21.411841
min,NaN,NaN,1.148593e+09,1.0,1.110000,11.000000,3.100000,1.000000,15.000000
25%,NaN,NaN,3.641713e+09,1.0,1.640000,60.750000,3.500000,10.250000,35.250000
50%,NaN,NaN,5.920755e+09,1.0,1.985000,118.000000,4.000000,16.000000,59.000000
75%,NaN,NaN,8.089709e+09,1.0,2.507500,225.750000,4.300000,20.000000,75.000000


### Property Attributes

In [262]:
# First 5 rows
prop_att_df.head()

,attribute_id,listing_id,bedrooms,bathrooms,floor_number,total_floors,year_built,is_rented,tenant_count,furnishing_status,metro_distance_km,parking_available,power_backup
0,1,L00001,5,3,9,9,2001,True,4,Furnished,7.26,False,True
1,2,L00002,2,2,19,29,2020,False,0,Unfurnished,4.84,True,False
2,3,L00003,2,3,8,26,2007,True,4,Semi-Furnished,4.92,False,False
3,4,L00004,3,3,25,10,2003,False,3,Furnished,1.20,False,True
4,5,L00005,2,2,15,16,2008,True,1,Semi-Furnished,7.90,False,False


In [263]:
# Last 5 rows
prop_att_df.tail()

,attribute_id,listing_id,bedrooms,bathrooms,floor_number,total_floors,year_built,is_rented,tenant_count,furnishing_status,metro_distance_km,parking_available,power_backup
21195,21196,L21196,5,4,3,6,2018,True,2,Semi-Furnished,6.21,True,True
21196,21197,L21197,5,1,0,33,2010,False,0,Unfurnished,2.16,False,True
21197,21198,L21198,5,2,7,29,1992,True,4,Semi-Furnished,6.60,True,True
21198,21199,L21199,5,3,20,21,2013,False,0,Semi-Furnished,1.76,False,False
21199,21200,L21200,4,2,9,30,2019,True,3,Furnished,12.12,False,False


In [264]:
# Basic Information
prop_att_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21200 entries, 0 to 21199
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   attribute_id       21200 non-null  int64  
 1   listing_id         21200 non-null  object 
 2   bedrooms           21200 non-null  int64  
 3   bathrooms          21200 non-null  int64  
 4   floor_number       21200 non-null  int64  
 5   total_floors       21200 non-null  int64  
 6   year_built         21200 non-null  int64  
 7   is_rented          21200 non-null  bool   
 8   tenant_count       21200 non-null  int64  
 9   furnishing_status  21200 non-null  object 
 10  metro_distance_km  21200 non-null  float64
 11  parking_available  21200 non-null  bool   
 12  power_backup       21200 non-null  bool   
dtypes: bool(3), float64(1), int64(7), object(2)
memory usage: 1.7+ MB


In [265]:
# Converting boolean to integer
var_list = ['is_rented', 'parking_available', 'power_backup']

prop_att_df[var_list] = prop_att_df[var_list].apply(lambda x: bool_to_int(x))
prop_att_df.head()

,attribute_id,listing_id,bedrooms,bathrooms,floor_number,total_floors,year_built,is_rented,tenant_count,furnishing_status,metro_distance_km,parking_available,power_backup
0,1,L00001,5,3,9,9,2001,1,4,Furnished,7.26,0,1
1,2,L00002,2,2,19,29,2020,0,0,Unfurnished,4.84,1,0
2,3,L00003,2,3,8,26,2007,1,4,Semi-Furnished,4.92,0,0
3,4,L00004,3,3,25,10,2003,0,3,Furnished,1.20,0,1
4,5,L00005,2,2,15,16,2008,1,1,Semi-Furnished,7.90,0,0


In [266]:
# Maintaining consistency with Column Name
prop_att_df.rename(columns={'listing_id': 'Listing_ID'}, inplace=True)
prop_att_df.head()

,attribute_id,Listing_ID,bedrooms,bathrooms,floor_number,total_floors,year_built,is_rented,tenant_count,furnishing_status,metro_distance_km,parking_available,power_backup
0,1,L00001,5,3,9,9,2001,1,4,Furnished,7.26,0,1
1,2,L00002,2,2,19,29,2020,0,0,Unfurnished,4.84,1,0
2,3,L00003,2,3,8,26,2007,1,4,Semi-Furnished,4.92,0,0
3,4,L00004,3,3,25,10,2003,0,3,Furnished,1.20,0,1
4,5,L00005,2,2,15,16,2008,1,1,Semi-Furnished,7.90,0,0


In [267]:
# Summary statistics
prop_att_df.describe(include='all')

,attribute_id,Listing_ID,bedrooms,bathrooms,floor_number,total_floors,year_built,is_rented,tenant_count,furnishing_status,metro_distance_km,parking_available,power_backup
count,21200.000000,21200,21200.000000,21200.000000,21200.000000,21200.000000,21200.000000,21200.000000,21200.000000,21200,21200.000000,21200.000000,21200.000000
unique,NaN,21200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN
top,NaN,L00001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unfurnished,NaN,NaN,NaN
freq,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7210,NaN,NaN,NaN
mean,10600.500000,NaN,3.003726,2.497877,9.636274,20.099009,2006.762830,0.498962,1.059340,NaN,7.333288,0.501226,0.490991
std,6120.057189,NaN,1.414059,1.119787,8.529010,11.138500,9.815126,0.500011,1.433155,NaN,4.287201,0.500010,0.499931
min,1.000000,NaN,1.000000,1.000000,0.000000,1.000000,1990.000000,0.000000,0.000000,NaN,0.100000,0.000000,0.000000
25%,5300.750000,NaN,2.000000,1.000000,3.000000,11.000000,1998.000000,0.000000,0.000000,NaN,3.630000,0.000000,0.000000
50%,10600.500000,NaN,3.000000,2.000000,7.000000,20.000000,2007.000000,0.000000,0.000000,NaN,7.175000,1.000000,0.000000
75%,15900.250000,NaN,4.000000,4.000000,15.000000,30.000000,2015.000000,1.000000,2.000000,NaN,11.010000,1.000000,1.000000


### Buyers

In [268]:
# First 5 rows
buyers_df.head()

,buyer_id,sale_id,buyer_type,payment_mode,loan_taken,loan_provider,loan_amount
0,1,L01179,End User,Cash,False,None,0
1,2,L00866,Investor,Cheque,False,None,0
2,3,L00102,Investor,Cheque,True,Axis,2317757
3,4,L00440,Investor,Bank Transfer,False,None,0
4,5,L00059,Investor,UPI,True,HDFC,4191221


In [269]:
# Last 5 rows
buyers_df.tail()

,buyer_id,sale_id,buyer_type,payment_mode,loan_taken,loan_provider,loan_amount
19995,19996,L00891,Investor,UPI,False,None,0
19996,19997,L00834,End User,UPI,False,None,0
19997,19998,L01188,Investor,UPI,False,None,0
19998,19999,L00905,Investor,Cheque,True,ICICI,3701968
19999,20000,L00526,Investor,Cash,True,SBI,3805678


In [270]:
# Convert boolean to integer in loan_taken column
buyers_df['loan_taken'] = buyers_df[['loan_taken']].apply(lambda x: bool_to_int(x))
buyers_df.head()

,buyer_id,sale_id,buyer_type,payment_mode,loan_taken,loan_provider,loan_amount
0,1,L01179,End User,Cash,0,None,0
1,2,L00866,Investor,Cheque,0,None,0
2,3,L00102,Investor,Cheque,1,Axis,2317757
3,4,L00440,Investor,Bank Transfer,0,None,0
4,5,L00059,Investor,UPI,1,HDFC,4191221


In [271]:
# Basic Information
buyers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   buyer_id       20000 non-null  int64 
 1   sale_id        20000 non-null  object
 2   buyer_type     20000 non-null  object
 3   payment_mode   20000 non-null  object
 4   loan_taken     20000 non-null  int64 
 5   loan_provider  10034 non-null  object
 6   loan_amount    20000 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 1.1+ MB


In [272]:
# Percentage of missing values
per_miss_val(buyers_df)

buyer_id          0.00
sale_id           0.00
buyer_type        0.00
payment_mode      0.00
loan_taken        0.00
loan_provider    49.83
loan_amount       0.00
dtype: float64

In [273]:
# Checking if the loan provider is null for the buyer who has not taken a loan
buyers_df[buyers_df['loan_taken'] == 0]['loan_provider'].unique()

array([None], dtype=object)

In [274]:
buyers_df[buyers_df['loan_amount'] == 0]['loan_provider'].unique()

array([None], dtype=object)

In [275]:
# Fill null values with "NA"
buyers_df['loan_provider'].fillna('NA', inplace=True)
buyers_df.head()

,buyer_id,sale_id,buyer_type,payment_mode,loan_taken,loan_provider,loan_amount
0,1,L01179,End User,Cash,0,NA,0
1,2,L00866,Investor,Cheque,0,NA,0
2,3,L00102,Investor,Cheque,1,Axis,2317757
3,4,L00440,Investor,Bank Transfer,0,NA,0
4,5,L00059,Investor,UPI,1,HDFC,4191221


In [276]:
# Percentage of null values
per_miss_val(buyers_df)

buyer_id         0.0
sale_id          0.0
buyer_type       0.0
payment_mode     0.0
loan_taken       0.0
loan_provider    0.0
loan_amount      0.0
dtype: float64

In [277]:
# Changing 'sale_id' column name to 'Listing_ID' to maintain consistency
buyers_df.rename(columns={'sale_id':'Listing_ID'}, inplace=True)
buyers_df.head()

,buyer_id,Listing_ID,buyer_type,payment_mode,loan_taken,loan_provider,loan_amount
0,1,L01179,End User,Cash,0,NA,0
1,2,L00866,Investor,Cheque,0,NA,0
2,3,L00102,Investor,Cheque,1,Axis,2317757
3,4,L00440,Investor,Bank Transfer,0,NA,0
4,5,L00059,Investor,UPI,1,HDFC,4191221


In [278]:
# Summary Statistics
buyers_df.describe(include='all')

,buyer_id,Listing_ID,buyer_type,payment_mode,loan_taken,loan_provider,loan_amount
count,20000.000000,20000,20000,20000,20000.00000,20000,2.000000e+04
unique,NaN,720,2,4,NaN,6,NaN
top,NaN,L01179,End User,Cash,NaN,NA,NaN
freq,NaN,28,10020,5088,NaN,9966,NaN
mean,10000.500000,NaN,NaN,NaN,0.50170,NaN,2.614323e+06
std,5773.647028,NaN,NaN,NaN,0.50001,NaN,3.252938e+06
min,1.000000,NaN,NaN,NaN,0.00000,NaN,0.000000e+00
25%,5000.750000,NaN,NaN,NaN,0.00000,NaN,0.000000e+00
50%,10000.500000,NaN,NaN,NaN,1.00000,NaN,5.272955e+05
75%,15000.250000,NaN,NaN,NaN,1.00000,NaN,5.235049e+06


### Listings

In [279]:
# First 5 rows
listings_df.head()

,Listing_ID,City,Property_Type,Price,Sqft,Date_Listed,Agent_ID,Latitude,Longitude
0,L00001,New York,Apartment,1.655144e+06,2753.009121,2023-05-06,A0015,33.965208,-69.861589
1,L00002,Los Angeles,Apartment,1.519141e+06,4966.988193,2023-02-14,A0038,42.547892,-90.277860
2,L00003,Houston,Apartment,1.624890e+05,1267.003959,2023-04-22,A0015,28.732327,-115.952982
3,L00004,Phoenix,Apartment,1.277016e+06,2128.014429,2024-01-02,A0042,26.403938,-74.771490
4,L00005,Phoenix,Townhouse,5.622970e+05,4178.997421,2023-10-29,A0018,39.425252,-83.917878


In [280]:
# Last 5 rows
listings_df.tail()

,Listing_ID,City,Property_Type,Price,Sqft,Date_Listed,Agent_ID,Latitude,Longitude
21195,L21196,Phoenix,House,709355.28,898.41,2023-12-31,A0021,25.456929,-88.189857
21196,L21197,Phoenix,Condo,3288420.13,4195.68,2023-07-23,A0005,40.926232,-80.167341
21197,L21198,Los Angeles,Apartment,1667441.24,8723.37,2024-06-30,A0003,46.083306,-115.396164
21198,L21199,Los Angeles,Townhouse,1648411.61,8412.71,2023-09-17,A0018,33.757737,-103.472564
21199,L21200,New York,Apartment,4991528.39,3994.92,2023-07-28,A0008,42.222541,-119.219408


In [281]:
# Percentage of missing values
print("Percentage of missing values")
per_miss_val(listings_df)

Percentage of missing values


Listing_ID       0.0
City             0.0
Property_Type    0.0
Price            0.0
Sqft             0.0
Date_Listed      0.0
Agent_ID         0.0
Latitude         0.0
Longitude        0.0
dtype: float64

In [283]:
# Basic Information
listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21200 entries, 0 to 21199
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Listing_ID     21200 non-null  object 
 1   City           21200 non-null  object 
 2   Property_Type  21200 non-null  object 
 3   Price          21200 non-null  float64
 4   Sqft           21200 non-null  float64
 5   Date_Listed    21200 non-null  object 
 6   Agent_ID       21200 non-null  object 
 7   Latitude       21200 non-null  float64
 8   Longitude      21200 non-null  float64
dtypes: float64(4), object(5)
memory usage: 1.5+ MB


In [286]:
# Summary Statistics
listings_df.describe(include='all')

,Listing_ID,City,Property_Type,Price,Sqft,Date_Listed,Agent_ID,Latitude,Longitude
count,21200,21200,21200,2.120000e+04,21200.000000,21200,21200,21200.000000,21200.000000
unique,21200,5,4,NaN,NaN,731,50,NaN,NaN
top,L00001,Los Angeles,Condo,NaN,NaN,2024-05-14,A0011,NaN,NaN
freq,1,4318,5363,NaN,NaN,46,463,NaN,NaN
mean,NaN,NaN,NaN,2.452708e+06,5113.376093,NaN,NaN,37.005657,-96.103864
std,NaN,NaN,NaN,1.419864e+06,2737.677578,NaN,NaN,6.934291,16.729760
min,NaN,NaN,NaN,1.000271e+05,500.005586,NaN,NaN,25.005494,-124.999086
25%,NaN,NaN,NaN,1.216681e+06,2768.277500,NaN,NaN,30.972635,-110.672440
50%,NaN,NaN,NaN,2.383803e+06,4974.845000,NaN,NaN,37.055855,-96.159857
75%,NaN,NaN,NaN,3.679127e+06,7488.895000,NaN,NaN,42.953084,-81.725564


### Sales

In [ ]:
# First 5 rows
sales_df.head()

,Listing_ID,Sale_Price,Date_Sold,Days_on_Market
0,L01179,9.255800e+05,2023-07-07,65.005560
1,L00866,1.054160e+05,2023-06-14,38.004620
2,L00102,1.825184e+06,2023-09-09,22.992622
3,L00440,1.932085e+06,2023-10-29,72.012274
4,L00059,7.765860e+05,2023-05-01,116.000152


In [ ]:
# Last 5 rows
sales_df.tail()

,Listing_ID,Sale_Price,Date_Sold,Days_on_Market
715,L00691,2.820920e+05,2024-06-18,101.992142
716,L00545,1.917896e+06,2023-12-16,33.006934
717,L01123,1.294670e+06,2023-06-21,45.996047
718,L00675,1.398092e+06,2024-04-15,49.998021
719,L00690,5.039320e+05,2023-05-13,118.997563


In [288]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Listing_ID      720 non-null    object 
 1   Sale_Price      720 non-null    float64
 2   Date_Sold       720 non-null    object 
 3   Days_on_Market  720 non-null    float64
dtypes: float64(2), object(2)
memory usage: 22.6+ KB


In [293]:
sales_df.describe()

,Sale_Price,Days_on_Market
count,7.200000e+02,720.000000
mean,1.077546e+06,61.642054
std,5.427902e+05,34.118649
min,1.054160e+05,4.987369
25%,6.031483e+05,30.751720
50%,1.076570e+06,62.992071
75%,1.532235e+06,91.019569
max,2.077611e+06,120.018183
